In [16]:
# Importing libraries and dependencies
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
from PIL import Image
from io import BytesIO
from matplotlib.offsetbox import (OffsetImage, AnnotationBbox)
from datetime import timedelta
import matplotlib.dates as mdates

#importing data

data = pd.read_csv(r'/Users/karolk/Python_Work/Data_Sets/Global_Data/DeFi_Global_DB.csv', index_col=0)
pd.set_option("display.max.columns", None)
pd.set_option("display.precision", 4)
pd.options.display.float_format = '{:,.2f}'.format

# remove 'camelot-v2' project from dataset
data = data[data['project'] != 'camelot-v2']

In [17]:
#modifying the data set to make sure 'date' is in datetime format and a new column for pools which are one-sided ETH pools
data['eth_one_sided'] = np.where((data['symbol'].str.contains('eth', case=False, na=False)) & (data['ilRisk'] == 'no'), True, False)
data['date'] = pd.to_datetime(data['date'])

#selecting only the colums needed to create the visualizations
time_data = data[['date', 'chain', 'project', 'pool', 'tvlUsd', 'stablecoin', 'outlier', 'exposure', 'eth_one_sided']]

#find the min of the date column
min_date = time_data['date'].min()

#change the smallest date in the date column to '2023-01-01'
time_data['date'] = time_data['date'].replace(min_date, '2023-01-01')

#create a variable called start of year which is the min date of the date column
start_of_year = time_data['date'].min()

#cleaning up the data by removing outliers and pools with no TVL
time_data = time_data[time_data['tvlUsd'] > 0]

# setting the date field in time to datetime
time_data['date'] = pd.to_datetime(time_data['date'])

#creating a list for liquid staking pools
liquid_stakers = ['lido', 'rocket-pool', 'coinbase-wrapped-staked-eth', 'stakewise', 'frax-ether', 'stafi', 'stakehound', 'stader', 'bitfrost-liquid-staking', 'binance-staked-eth', 'swell', 'ankr']

#create a new column in 'time_data' that is 'True' if the project is in the liquid_stakers list
time_data['liquid_staker'] = time_data['project'].isin(liquid_stakers)



/var/folders/gn/b1hbt5mx3mg2y8p3tgg7g7xh0000gn/T/ipykernel_2021/3598419691.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  time_data['date'] = time_data['date'].replace(min_date, '2023-01-01')


In [30]:
#keep only the date where date=2023-11-15
time_data = time_data[time_data['date'] == '2023-11-15']

#create a new table summing the TVL per chain where liquid staking is 'False'
tvl_per_chain = time_data[time_data['liquid_staker'] == False].groupby(['chain'])['tvlUsd'].sum().reset_index()
# sort by TVL
tvl_per_chain = tvl_per_chain.sort_values(by=['tvlUsd'], ascending=False)

#get the time data for rows where chain = 'BSC' and project = 'venus-core-pool'
time_data[(time_data['chain'] == 'BSC') & (time_data['project'] == 'venus-core-pool')]

,date,chain,project,pool,tvlUsd,stablecoin,outlier,exposure,eth_one_sided,liquid_staker
id,,,,,,,,,,
27,2023-11-15,BSC,venus-core-pool,87c8ee0d-b812-47c1-803f-f91a3907079e,"267,311,911.00",False,False,single,False,False
38,2023-11-15,BSC,venus-core-pool,747b58ab-aefd-42e1-a312-01ad5a0ab7f5,"167,542,628.00",False,False,single,False,False
108,2023-11-15,BSC,venus-core-pool,de8928ad-d03a-423d-92d7-3c4648e3ffd2,"54,908,088.00",False,False,single,True,False
162,2023-11-15,BSC,venus-core-pool,9f3a6015-5045-4471-ba65-ad3dc7c38269,"35,204,962.00",True,False,single,False,False
185,2023-11-15,BSC,venus-core-pool,b8872d86-c39e-423c-83dd-8b32f7eea91d,"29,586,874.00",False,False,single,True,False
223,2023-11-15,BSC,venus-core-pool,88472ba3-f1e9-4da6-89da-eb12cf07e151,"22,543,702.00",False,False,single,False,False
289,2023-11-15,BSC,venus-core-pool,89eba1e5-1b1b-47b6-958b-38138a04c244,"15,909,089.00",True,False,single,False,False
490,2023-11-15,BSC,venus-core-pool,bd3e33f0-fa0f-4c4c-97d7-90f267808355,"7,582,566.00",False,False,single,False,False
540,2023-11-15,BSC,venus-core-pool,0204c8ff-0805-4515-a27e-742d23a15719,"6,627,266.00",False,False,single,False,False


In [27]:
# create a table that sums the TVL per project per chain where liquid staking is 'False'
tvl_per_project = time_data[time_data['liquid_staker'] == False].groupby(['chain', 'project'])['tvlUsd'].sum().reset_index()

# sort by TVL
tvl_per_project = tvl_per_project.sort_values(by=['tvlUsd'], ascending=False)

# show the table where Chain = 'Tron'
tvl_per_project[tvl_per_project['chain'] == 'BSC']

,chain,project,tvlUsd
196,BSC,venus-core-pool,"638,533,079.00"
182,BSC,pancakeswap-amm-v3,"232,582,131.00"
181,BSC,pancakeswap-amm,"91,582,825.00"
153,BSC,alpaca-leveraged-yield-farming,"81,243,383.00"
171,BSC,helio-protocol,"53,141,445.00"
160,BSC,biswap-v2,"45,277,988.00"
185,BSC,radiant-v2,"32,850,401.00"
191,BSC,thorchain,"29,697,651.00"
198,BSC,wombat-exchange,"29,536,930.00"
167,BSC,gamma,"26,598,359.00"
